In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import pymongo

In [2]:
!which chromedriver

/usr/local/bin/chromedriver


In [3]:
marsclient = pymongo.MongoClient("mongodb://localhost:27017/")
marsdb = marsclient["mars_app"]
marscol = marsdb["marshemi"]

In [4]:
executable_path = {'executable_path':'/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless = False )

In [5]:
# Visit the mars nasa news site
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)

False

In [6]:
html = browser.html
news_soup = BeautifulSoup(html, 'html.parser')
slide_elem = news_soup.select_one('ul.item_list li.slide')

There are two methods used to find tags and attributes with BeautifulSoup:
.find() is used when we want only the first class and attribute we’ve specified.
.find_all() is used when we want to retrieve all of the tags and attributes.
For example, if we were to use .find_all() instead of .find() when pulling the summary, we would retrieve all of the summaries on the page instead of just the first one.

In [7]:
news_title = slide_elem.find('div', class_='content_title').get_text()

In [8]:
print(news_title)

NASA's Curiosity Mars Rover Takes a New Selfie Before Record Climb


In [9]:
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
print(news_p)

Along with capturing an image before its steepest ascent ever, the robotic explorer filmed its "selfie stick," or robotic arm, in action.


In [10]:
news_date = slide_elem.find('div',class_='list_date').get_text()
print(news_date)

March 20, 2020


###  Featured Images

In [11]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

"ID" is UNIQUE in the webpage, and can only be used once throughout 
the entire page 

In [12]:
full_image_elem = browser.find_by_id('full_image')
full_image_elem.click()

In [13]:
browser.is_element_present_by_text('more info', wait_time = 1)
more_info_elem = browser.find_link_by_partial_text('more info')
more_info_elem.click()

/Users/pegkhiev/opt/anaconda3/lib/python3.7/site-packages/splinter/driver/webdriver/__init__.py:528: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


In [14]:
# FutureWarning: browser.find_link_by_partial_text is deprecated. 
# Use browser.links.find_by_partial_text instead.
#   FutureWarning,

# Parse the resulting html with soup
html = browser.html
img_soup = BeautifulSoup(html, 'html.parser')

In [15]:
#Find the relevant image url
image_url_rel = img_soup.select_one('figure.lede a img').get('src')
image_url_rel

'/spaceimages/images/largesize/PIA16613_hires.jpg'

In [16]:
img_url = f"https://jpl.nasa.gov{image_url_rel}"
img_url

'https://jpl.nasa.gov/spaceimages/images/largesize/PIA16613_hires.jpg'

In [17]:
df = pd.read_html('https://space-facts.com/mars/')[0]
df.columns = ['description','mars']
df.set_index('description', inplace = True)
df

# By specifying an index of 0, we’re telling Pandas to pull only the 
# first table it encounters. Then, it turns the table into a DataFrame.

,mars
description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [18]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>mars</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\n

## CHALLENGE


In [19]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [20]:
mars_hemi=[]
def gethemi(x):
    dict = {}
    hemispheres = browser.find_by_css('h3')[x]
    hemispheres.click()
    hemi_html = browser.html
    hemi = BeautifulSoup(hemi_html, 'html.parser')
    hemi_url_rel = hemi.find('img', class_= 'wide-image').get('src')
    hemi_title= hemi.find('h2',class_='title').text
    hemi_url = f"https://astrogeology.usgs.gov{hemi_url_rel}"
    dict["img_url"] = hemi_url
    dict['title']= hemi_title
    browser.back()
    return mars_hemi.append(dict)

In [21]:
gethemi(0)

In [22]:
gethemi(1)

In [23]:
gethemi(2)

In [24]:
gethemi(3)

In [25]:
mars_hemi

[{'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [26]:
marscol.insert_many(mars_hemi)

In [27]:
browser.quit()